In [112]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
#plt.style.use('fivethirtyeight')
import xgboost as xgb
import sklearn
import random
from NecessaryModules.getData import getData


In [113]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [114]:
filename = 'Dataset/ionosphere.csv'
dataset,label=getData(filename)
# dataset=pd.DataFrame(dataset) #from dataset_list to dataset_dataframe after suffling
print(dataset)
print(label)

[[1.0, 0.0, 0.99539, -0.05889, 0.85243, 0.02306, 0.83398, -0.37708, 1.0, 0.0376, 0.85243, -0.17755, 0.59755, -0.44945, 0.60536, -0.38223, 0.84356, -0.38542, 0.58212, -0.32192, 0.56971, -0.29674, 0.36946, -0.47357, 0.56811, -0.51171, 0.41078, -0.46168, 0.21266, -0.3409, 0.42267, -0.54487, 0.18641, -0.453], [1.0, 0.0, 1.0, -0.18829, 0.93035, -0.36156, -0.10868, -0.93597, 1.0, -0.04549, 0.50874, -0.67743, 0.34432, -0.69707, -0.51685, -0.97515, 0.05499, -0.62237, 0.33109, -1.0, -0.13151, -0.453, -0.18056, -0.35734, -0.20332, -0.26569, -0.20468, -0.18401, -0.1904, -0.11593, -0.16626, -0.06288, -0.13738, -0.02447], [1.0, 0.0, 1.0, -0.03365, 1.0, 0.00485, 1.0, -0.12062, 0.88965, 0.01198, 0.73082, 0.05346, 0.85443, 0.00827, 0.54591, 0.00299, 0.83775, -0.13644, 0.75535, -0.0854, 0.70887, -0.27502, 0.43385, -0.12062, 0.57528, -0.4022, 0.58984, -0.22145, 0.431, -0.17365, 0.60436, -0.2418, 0.56045, -0.38238], [1.0, 0.0, 1.0, -0.45161, 1.0, 1.0, 0.71216, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.

In [115]:
import numpy as np


def split_data(dataset, label):
    # train-test split
    indices = np.random.permutation(len(dataset))
    training_idx, test_idx = indices[:221], indices[221:]
    x_train = [dataset[i] for i in training_idx]
    x_test = [dataset[i] for i in test_idx]
    y_train = [label[i] for i in training_idx]
    y_test = [label[i] for i in test_idx]

    return x_train, y_train, x_test, y_test

# RFE implement

In [116]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR


estimator = SVR(kernel="linear")
model = RFE(estimator, n_features_to_select=10, step=1)
model.fit(dataset,label)



RFE(estimator=SVR(kernel='linear'), n_features_to_select=10)

In [117]:

compressed_dataset_X= model.transform(dataset) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X))
print(len(compressed_dataset_X[0]))
print(compressed_dataset_X)

<class 'numpy.ndarray'>
10
[[ 1.0000e+00  9.9539e-01 -5.8890e-02 ... -5.1171e-01  4.1078e-01
   4.2267e-01]
 [ 1.0000e+00  1.0000e+00 -1.8829e-01 ... -2.6569e-01 -2.0468e-01
  -1.6626e-01]
 [ 1.0000e+00  1.0000e+00 -3.3650e-02 ... -4.0220e-01  5.8984e-01
   6.0436e-01]
 ...
 [ 1.0000e+00  9.4701e-01 -3.4000e-04 ...  3.1930e-02  9.2489e-01
   9.2459e-01]
 [ 1.0000e+00  9.0608e-01 -1.6570e-02 ... -2.0990e-02  8.9147e-01
   9.6022e-01]
 [ 1.0000e+00  8.4710e-01  1.3533e-01 ... -1.5114e-01  8.1147e-01
   7.5747e-01]]


In [118]:
# from NecessaryModules.splitData import split_data
X_train, y_train, X_comb, y_comb = split_data(dataset,label)
print(len(y_comb))

130


In [119]:
X_valid, X_test, y_valid, y_test = train_test_split(X_comb, y_comb, test_size=0.50, random_state=42)
print(len(X_valid))

65


In [120]:
compressed_dataset_X_train= model.transform(X_train) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_train))
print(len(compressed_dataset_X_train))
print(compressed_dataset_X_train)

compressed_dataset_X_valid= model.transform(X_valid) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_valid))
print(len(compressed_dataset_X_valid))
print(compressed_dataset_X_valid)

compressed_dataset_X_test= model.transform(X_test) # --transform-- the original "dataset" Dataframe into a lower-dimensional space based on the model's learned parameters
print(type(compressed_dataset_X_test))
print(len(compressed_dataset_X_test))
print(compressed_dataset_X_test)

<class 'numpy.ndarray'>
221
[[ 0.       1.      -1.      ... -1.       1.       1.     ]
 [ 1.       1.      -1.      ...  0.       1.       1.     ]
 [ 1.       0.93658  0.35107 ... -0.84951 -0.04578  0.54827]
 ...
 [ 1.       1.       1.      ...  0.       1.       1.     ]
 [ 1.       0.98455 -0.02736 ... -0.08341  0.96653  0.97132]
 [ 1.       1.       0.23395 ...  0.03591 -0.71731 -0.51251]]
<class 'numpy.ndarray'>
65
[[ 1.0000e+00  1.0135e-01  1.0811e-01  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00 -1.0000e+00  1.0000e+00  0.0000e+00]
 [ 1.0000e+00  1.0000e+00 -1.0000e+00  1.0000e+00  1.0000e+00 -1.0000e+00
   1.0000e+00 -1.0000e+00  1.0000e+00 -1.0000e+00]
 [ 1.0000e+00  9.8002e-01  7.5000e-04  1.0000e+00 -7.5000e-04  9.7738e-01
   5.5240e-02  8.1070e-02  9.6709e-01  9.8190e-01]
 [ 1.0000e+00  3.9286e-01  5.2381e-01 -7.8824e-01 -7.6378e-01 -5.1830e-01
   4.4961e-01  7.0238e-01  5.1971e-01 -5.4891e-01]
 [ 1.0000e+00  8.7578e-01  3.7270e-02  8.9951e-01  5.1000e-03  8.4182e-01

# reshape the data 

In [121]:
compressed_dataset_X_valid.shape

(65, 10)

In [122]:
X_train_arr=compressed_dataset_X_train.reshape(compressed_dataset_X_train.shape[0],compressed_dataset_X_train.shape[1],1)
X_train_arr.shape

(221, 10, 1)

In [123]:
# X_valid_arr=compressed_dataset_X_valid.to_numpy()
X_valid_arr=compressed_dataset_X_valid.reshape(compressed_dataset_X_valid.shape[0],compressed_dataset_X_valid.shape[1],1)
X_valid_arr.shape

(65, 10, 1)

In [124]:
X_test_arr=compressed_dataset_X_test.reshape(compressed_dataset_X_test.shape[0],compressed_dataset_X_test.shape[1],1)
X_test_arr.shape

(65, 10, 1)

In [125]:
# for "y"

y_train_arr=np.array(y_train)
y_train_arr=y_train_arr.reshape(y_train_arr.shape[0],1)

y_valid_arr=np.array(y_valid)
y_valid_arr=y_valid_arr.reshape(y_valid_arr.shape[0],1)

y_test_arr=np.array(y_test)
y_test_arr=y_test_arr.reshape(y_test_arr.shape[0],1)

In [126]:
y_test_arr.shape

(65, 1)

# one hot encode

In [127]:
from keras.utils import to_categorical
# one hot encoding
y_train_arr = to_categorical(y_train, 7)
y_valid_arr = to_categorical(y_valid, 7)
y_test_arr = to_categorical(y_test, 7)

In [128]:
y_test_arr.shape

(65, 7)

# model

In [129]:

# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D,LSTM


In [130]:
import tensorflow as tf
callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)

In [131]:
# Global value
verbose, epochs, batch_size = 1, 100, 20


In [132]:
# summarize scores
def summarize_results(scores):
 print(scores)
 m, s = mean(scores), std(scores)
 print('Test Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 

# Accuracy measure

**tunned model**

In [133]:
# fit and evaluate a model
import optuna
from matplotlib import pyplot as plt

def optimize_model2(trainX, trainy,validX,validy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], 7
    def objective_model2(trial):
    # Define hyperparameters to be tuned
     
     filters = trial.suggest_int('filters', 32, 128)
     kernel_size = trial.suggest_int('kernel_size', 2, 5)
     dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    #  dense_units = trial.suggest_int('dense_units', 50, 200)
     pool_size=trial.suggest_int('pool_size',2,4)
     
     
     # Build the CNN model
     model = Sequential()
     model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(n_timesteps, n_features)))
     model.add(MaxPooling1D(pool_size=pool_size))
     model.add(LSTM(units=filters)) # remember the important features
     model.add(Dense(100, activation='relu'))
     model.add(Dense(filters, activation='relu')) 
     model.add(Dropout(dropout_rate))#for regularization
     model.add(Dense(n_outputs, activation='softmax'))
 
     # Compile the model
     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
     # Fit the model
     history = model.fit(trainX, trainy, validation_data=(validX, validy), epochs=epochs,callbacks=[callback],batch_size=batch_size, verbose=verbose)
    
    # Return the validation accuracy as the objective value
     return history.history['val_accuracy'][-1]
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_model2, n_trials= 10, 
                   #timeout=600
                   )#The maximum time (in seconds) allowed for the optimization process to run.
    # Get the best hyperparameters
    best_params = study.best_params
    print("Best Hyperparameters:", best_params)
    fig_study = optuna.visualization.plot_parallel_coordinate(study, params=["filters", "kernel_size","dropout_rate","pool_size"])
    fig_study.show()

    return best_params
 



In [134]:

best_params=optimize_model2(X_train_arr, y_train_arr, X_valid_arr, y_valid_arr)

[I 2024-02-11 01:02:29,831] A new study created in memory with name: no-name-53b0f7c1-5df9-4a8c-98f7-215fc9d166b8


Epoch 1/100
12/12 [==============================] - 4s 144ms/step - loss: 1.9081 - accuracy: 0.5747 - val_loss: 1.8581 - val_accuracy: 0.5077
Epoch 2/100
12/12 [==============================] - 0s 8ms/step - loss: 1.7739 - accuracy: 0.6652 - val_loss: 1.6884 - val_accuracy: 0.5077
Epoch 3/100
12/12 [==============================] - 0s 12ms/step - loss: 1.4729 - accuracy: 0.6652 - val_loss: 1.3328 - val_accuracy: 0.5077
Epoch 4/100
12/12 [==============================] - 0s 23ms/step - loss: 1.0436 - accuracy: 0.6561 - val_loss: 0.9620 - val_accuracy: 0.5077
Epoch 5/100
12/12 [==============================] - 0s 21ms/step - loss: 0.7370 - accuracy: 0.6878 - val_loss: 0.6883 - val_accuracy: 0.5846
Epoch 6/100
12/12 [==============================] - 0s 7ms/step - loss: 0.5946 - accuracy: 0.7014 - val_loss: 0.6889 - val_accuracy: 0.6000
Epoch 7/100
12/12 [==============================] - 0s 7ms/step - loss: 0.5621 - accuracy: 0.7557 - val_loss: 0.5946 - val_accuracy: 0.6769
Epoch 8/

[I 2024-02-11 01:02:36,895] Trial 0 finished with value: 0.8461538553237915 and parameters: {'filters': 32, 'kernel_size': 4, 'dropout_rate': 0.25577374917753853, 'pool_size': 3}. Best is trial 0 with value: 0.8461538553237915.


Epoch 1/100
12/12 [==============================] - 3s 52ms/step - loss: 1.8706 - accuracy: 0.6471 - val_loss: 1.7449 - val_accuracy: 0.5846
Epoch 2/100
12/12 [==============================] - 0s 9ms/step - loss: 1.5565 - accuracy: 0.7104 - val_loss: 1.2776 - val_accuracy: 0.5538
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 0.9758 - accuracy: 0.6833 - val_loss: 0.7963 - val_accuracy: 0.5538
Epoch 4/100
12/12 [==============================] - 0s 8ms/step - loss: 0.6931 - accuracy: 0.6787 - val_loss: 0.6987 - val_accuracy: 0.6000
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 0.5307 - accuracy: 0.7873 - val_loss: 0.5779 - val_accuracy: 0.6923
Epoch 6/100
12/12 [==============================] - 0s 10ms/step - loss: 0.4255 - accuracy: 0.8371 - val_loss: 0.5531 - val_accuracy: 0.7385
Epoch 7/100
12/12 [==============================] - 0s 9ms/step - loss: 0.3822 - accuracy: 0.8552 - val_loss: 0.5775 - val_accuracy: 0.7692
Epoch 8/100

[I 2024-02-11 01:02:41,756] Trial 1 finished with value: 0.8307692408561707 and parameters: {'filters': 60, 'kernel_size': 4, 'dropout_rate': 0.43398262049841263, 'pool_size': 4}. Best is trial 0 with value: 0.8461538553237915.


...

Epoch 1/100
12/12 [==============================] - 3s 50ms/step - loss: 1.7127 - accuracy: 0.6335 - val_loss: 1.2793 - val_accuracy: 0.5077
Epoch 2/100
12/12 [==============================] - 0s 8ms/step - loss: 0.9028 - accuracy: 0.6652 - val_loss: 0.7350 - val_accuracy: 0.5077
Epoch 3/100
12/12 [==============================] - 0s 8ms/step - loss: 0.6955 - accuracy: 0.5792 - val_loss: 0.8559 - val_accuracy: 0.5077
Epoch 4/100
12/12 [==============================] - 0s 8ms/step - loss: 0.6957 - accuracy: 0.6471 - val_loss: 0.6353 - val_accuracy: 0.6308
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 0.5447 - accuracy: 0.7376 - val_loss: 0.6895 - val_accuracy: 0.6154
Epoch 6/100
12/12 [==============================] - 0s 8ms/step - loss: 0.4717 - accuracy: 0.7873 - val_loss: 0.6627 - val_accuracy: 0.6462
Epoch 7/100
12/12 [==============================] - 0s 8ms/step - loss: 0.4258 - accuracy: 0.8371 - val_loss: 0.5657 - val_accuracy: 0.7077
Epoch 8/100


[I 2024-02-11 01:03:21,873] Trial 8 finished with value: 0.7692307829856873 and parameters: {'filters': 108, 'kernel_size': 2, 'dropout_rate': 0.13817970452614026, 'pool_size': 4}. Best is trial 4 with value: 0.9076923131942749.


Epoch 1/100
12/12 [==============================] - 3s 47ms/step - loss: 1.8492 - accuracy: 0.5701 - val_loss: 1.6829 - val_accuracy: 0.5077
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 1.4180 - accuracy: 0.6697 - val_loss: 1.1385 - val_accuracy: 0.5077
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 0.8882 - accuracy: 0.6652 - val_loss: 0.8327 - val_accuracy: 0.5077
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 0.6757 - accuracy: 0.6606 - val_loss: 0.7260 - val_accuracy: 0.5077
Epoch 5/100
12/12 [==============================] - 0s 7ms/step - loss: 0.6549 - accuracy: 0.6606 - val_loss: 0.6859 - val_accuracy: 0.5846
Epoch 6/100
12/12 [==============================] - 0s 6ms/step - loss: 0.5969 - accuracy: 0.7104 - val_loss: 0.6145 - val_accuracy: 0.6462
Epoch 7/100
12/12 [==============================] - 0s 7ms/step - loss: 0.4957 - accuracy: 0.7828 - val_loss: 0.7721 - val_accuracy: 0.6000
Epoch 8/100


[I 2024-02-11 01:03:28,838] Trial 9 finished with value: 0.800000011920929 and parameters: {'filters': 61, 'kernel_size': 2, 'dropout_rate': 0.217410822325768, 'pool_size': 4}. Best is trial 4 with value: 0.9076923131942749.


Best Hyperparameters: {'filters': 88, 'kernel_size': 5, 'dropout_rate': 0.3472139860540476, 'pool_size': 4}


In [135]:
best_params

{'filters': 88,
 'kernel_size': 5,
 'dropout_rate': 0.3472139860540476,
 'pool_size': 4}

In [136]:
# fit and evaluate a model
def evaluate_model_1(trainX, trainy,validX,validy, testX, testy):
#  verbose, epochs, batch_size = 1, 1500, 100
 n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], 7
 model = Sequential()
 model.add(Conv1D(filters=best_params['filters'], kernel_size=best_params['kernel_size'], activation='relu', input_shape=(n_timesteps,n_features)))
 model.add(MaxPooling1D(pool_size=best_params['pool_size']))
 model.add(LSTM(units=best_params['filters']))
 model.add(Dense(100, activation='relu'))
 model.add(Dense(best_params['filters'], activation='relu'))
 model.add(Dropout(best_params['dropout_rate']))
 model.add(Dense(n_outputs, activation='softmax')) # "softmax" for multi-class
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # fit network
 history=model.fit(trainX, trainy, validation_data=(validX, validy), epochs=epochs, batch_size=batch_size,callbacks=[callback], verbose=verbose) 
 # evaluate model
 _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
 # '_,'-->a variable is being used for a specific purpose, but its value is not of interest or not used in the subsequent code.
 return accuracy,history

In [137]:

score_1,history_1= evaluate_model_1(X_train_arr, y_train_arr, X_valid_arr, y_valid_arr, X_test_arr, y_test_arr)
score_1 = score_1 * 100.0
summarize_results(score_1)

Epoch 1/100
12/12 [==============================] - 3s 50ms/step - loss: 1.8552 - accuracy: 0.6787 - val_loss: 1.6898 - val_accuracy: 0.5846
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 1.3993 - accuracy: 0.6878 - val_loss: 1.0545 - val_accuracy: 0.5077
Epoch 3/100
12/12 [==============================] - 0s 9ms/step - loss: 0.7729 - accuracy: 0.6878 - val_loss: 0.6514 - val_accuracy: 0.6154
Epoch 4/100
12/12 [==============================] - 0s 9ms/step - loss: 0.6337 - accuracy: 0.6968 - val_loss: 0.6840 - val_accuracy: 0.6308
Epoch 5/100
12/12 [==============================] - 0s 9ms/step - loss: 0.4800 - accuracy: 0.7919 - val_loss: 0.5800 - val_accuracy: 0.7077
Epoch 6/100
12/12 [==============================] - 0s 9ms/step - loss: 0.3985 - accuracy: 0.8959 - val_loss: 0.5375 - val_accuracy: 0.8000
Epoch 7/100
12/12 [==============================] - 0s 8ms/step - loss: 0.3196 - accuracy: 0.9005 - val_loss: 0.5798 - val_accuracy: 0.8000
Epoch 8/100


**defined-model**

In [138]:
# import pickle

# # fit and evaluate a model
# def evaluate_model_2(trainX, trainy,validX,validy, testX, testy):
# #  verbose, epochs, batch_size = 1, 1500, 100
#  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], 7

#  model = Sequential()
#  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
#  model.add(MaxPooling1D(pool_size=2))
#  model.add(LSTM(units=64)) # remember the important features
#  model.add(Dense(100, activation='relu'))
#  model.add(Dense(64, activation='relu')) 
#  model.add(Dropout(0.5))#for regularization
#  model.add(Dense(n_outputs, activation='softmax'))
#  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#  # fit network
#  history=model.fit(trainX, trainy, validation_data=(validX, validy), epochs=epochs, batch_size=batch_size,callbacks=[callback], verbose=verbose)
#  #summarize_mode
#  print(model.summary())

#  #load-model
#  pickle.dump(model, open("../all_pkl_file/all_tuned_model/PCA_model2_fitted_2.pkl", "wb"))
 
#  # evaluate model
#  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
#  # '_,'-->a variable is being used for a specific purpose, but its value is not of interest or not used in the subsequent code.
#  return accuracy,history

In [139]:
# score_2,history_2= evaluate_model_2(X_train_arr, y_train_arr, X_valid_arr, y_valid_arr, X_test_arr, y_test_arr)
# score_2 = score_2 * 100.0
# summarize_results(score_2)

# Evaluation

In [140]:
# # plot loss-curve_2
# pyplot.plot(history_2.history['loss'], label='training')
# pyplot.plot(history_2.history['val_loss'], label='validation')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.title('Loss-curve')
# pyplot.legend()
# pyplot.show()

# Confusion-matrix

In [141]:
# # Load the encoder model from the saved file
# with open("../all_pkl_file/all_tuned_model/PCA_model2_fitted_2.pkl", "rb") as file: #"rb"= read mode
#     model = pickle.load(file)

# model

**confusionmatrix**

In [142]:
#  #confusion_matrix

# from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix


# predictions=model.predict(X_test_arr)
# print(y_test_arr)
# print(len(y_test_arr),',',len(y_test_arr[0]))
# print(predictions)
# print(len(predictions),',',len(predictions[0]))

# for i in range(len(predictions)):
#     for j in range(len(predictions[0])):
#         if predictions[i][j]==max(predictions[i]):
#             predictions[i][j]=1
#         else: predictions[i][j]=0

# # reverse one-hot encoding for fit into confusion matrix
# y_test_cm=y_test_arr.argmax(axis=1)
# print(y_test_cm)
# predictions_cm=predictions.argmax(axis=1)
# print(predictions_cm)

# cm = confusion_matrix(y_test_cm, predictions_cm, labels=[0,1,2,3,4,5,6])
# disp = ConfusionMatrixDisplay(confusion_matrix=cm,
#                             display_labels=[0,1,2,3,4,5,6])
# disp.plot()
# plt.show()


# Evaluation

In [143]:
# # plot loss-curve
# pyplot.plot(history_1.history['loss'], label='training')
# pyplot.plot(history_1.history['val_loss'], label='validation')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.title('Loss-curve')
# pyplot.legend()
# pyplot.show()

In [144]:
# # plot training history
# pyplot.plot(history.history['accuracy'], label='train')
# pyplot.plot(history.history['val_accuracy'], label='test')
# plt.xlabel('epoch')
# plt.ylabel('accuracy')
# plt.title('Accuracy-curve')
# pyplot.legend()
# pyplot.show()

# ROC curve

In [145]:
# # Load the encoder model from the saved file
# with open("../all_pkl_file/all_tuned_model/PCA_model2_fitted_2.pkl", "rb") as file: #"rb"= read mode
#     model = pickle.load(file)

# model

In [146]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, auc

In [147]:
# predictions=model.predict(X_test_arr)
# for i in range(len(predictions)):
#     for j in range(len(predictions[0])):
#         if predictions[i][j]==max(predictions[i]):
#             predictions[i][j]=1
#         else: predictions[i][j]=0
# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()

# for i in range(7):
#     fpr[i], tpr[i], _ = roc_curve(y_test_arr[:, i], predictions[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

In [148]:
# # Plot ROC curve for each class
# plt.figure()
# for i in range(7):
#     plt.plot(fpr[i], tpr[i], label=f'class {i}(area = {roc_auc[i]:0.2f})')
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.legend(loc='lower right')
# plt.show()